<a href="https://colab.research.google.com/github/scienclick/ScikitLearnPipeline4NLP/blob/master/crackdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# <editor-fold desc="Import">
import torch
from torchsummary import summary
import torch.nn as nn
from torchtext.legacy import data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import random
import re
import time

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import f1_score
from torch.utils.data import TensorDataset,DataLoader
from collections import Counter
from symspellpy import SymSpell,Verbosity
import torch.nn.functional as F

lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words("english")
from util.dependencies import tokenize
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from torchtext.legacy import data

lemmatizer = WordNetLemmatizer()
stop_words = nltk.corpus.stopwords.words("english")
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau,StepLR,MultiStepLR
# </editor-fold>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())  # normalize case and remove punctuation
    tokens = word_tokenize(text)  # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # lemmatize andremove stop words
    return tokens

def classification_rep(X_test,y_test):
    x_test=torch.Tensor(transformed.transform(X_test).toarray()).to(device)
    with torch.no_grad():
        preds=iris_model(x_test).cpu()
        preds.shape
        t,i=torch.max(nn.functional.log_softmax(preds,dim=1),1)
        # t,i=torch.max(preds,1)
        print(classification_report(y_test, i))
        return i.numpy()
    
def plot_confusion_matrix_(data,predictions):
    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    import matplotlib.pyplot as plt
    category_id_df = data[['Activity', "label"]].drop_duplicates().sort_values('label').set_index("label")
    a_dict=category_id_df.to_dict()["Activity"]
    col=list(a_dict.keys())
    col_=[a_dict[i] for i in col]
    c=col_
    y_test_=[a_dict[i] for i in y_test]
    predictions_=[a_dict[i] for i in predictions]
    conf_mat = confusion_matrix(y_test_, predictions_,labels=c)
    fig, ax = plt.subplots(figsize=(10,10))
    plt_recall=100*conf_mat // conf_mat.sum(axis=1)
    plt_prec=100*conf_mat // conf_mat.sum(axis=0)
    plt.figure()
    sns.heatmap(plt_recall, annot=True, fmt='d',xticklabels=c, yticklabels=c).set_title('Recall')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.figure()
    sns.heatmap(plt_prec, annot=True, fmt='d',xticklabels=c, yticklabels=c).set_title('Precission')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.figure()
    sns.heatmap(conf_mat, annot=True, fmt='d',xticklabels=c, yticklabels=c).set_title('confusion matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    
def train(model, iterator, optimizer, criterion):
    # iterator=train_iterator
    # batch.text.shape
    # predictions.shape
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text)

        # loss = nn.functional.nll_loss(nn.functional.log_softmax(predictions), batch.label)
        loss = criterion(predictions, batch.label)

        acc = categorical_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)
def train_(model, iterator, optimizer, criterion):
    # iterator=train_iterator
    # batch.text.shape
    # predictions.shape
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text)

        loss = nn.functional.nll_loss(nn.functional.log_softmax(predictions), batch.label)
        # loss = criterion(predictions, batch.label)

        acc = categorical_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)
def evaluate(model, iterator, criterion):
    # iterator=valid_iterator
    # iterator=train_iterator
    # batch.text


    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            # print(batch.text)
            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)

            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

## scikit-learn pytorch combination


In [ ]:
data = pd.read_excel("train.xlsx")

X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['label'], random_state=444,test_size=.00001)
print('rows in the original data set: {}'.format(data.shape[0]))
print('rows in the training set: {}'.format(X_train.shape[0]))
print('rows in the test set: {}'.format(X_test.shape[0]))

In [ ]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize,ngram_range=(1,3),min_df=4,max_df=.8)),
        ('tfidf', TfidfTransformer()),
    ])
    return pipeline
proc = model_pipeline()
transformed=proc.fit(X_train);
vals=transformed.transform(X_train).toarray()
features=proc.steps[0][1].get_feature_names()
frequency_matrix_count_1 = pd.DataFrame(vals, columns=features)
frequency_matrix_count_1

In [ ]:
iris=TensorDataset(torch.FloatTensor(frequency_matrix_count_1.values),torch.LongTensor(y_train))
iris_loader=DataLoader(iris,batch_size=64,shuffle=True)

In [ ]:
class Model(nn.Module):
    def __init__(self, in_,h1,h2, out_):
        super().__init__()
        self.fc1=nn.Linear(in_,h1)
        self.fc2=nn.Linear(h1,h2)
        self.fc3=nn.Linear(h2,out_)

    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
class Model2(nn.Module):
    def __init__(self, in_,h1,h2,h3, out_,p=0.5):
        super().__init__()
        self.dropout=nn.Dropout(p=p)
        self.fc1=nn.Linear(in_,h1)
        self.fc2=nn.Linear(h1,h2)
        self.fc3=nn.Linear(h2,h3)
        self.fc4=nn.Linear(h3,out_)

    def forward(self,x):
        x=self.dropout(F.relu(self.fc1(x)))
        x=self.dropout(F.relu(self.fc2(x)))
        x=self.dropout(F.relu(self.fc3(x)))
        x=self.fc4(x)
        return x

iris_model=Model(frequency_matrix_count_1.shape[1],128,64,25)
iris_model=Model2(frequency_matrix_count_1.shape[1],128,64,32,25,p=0)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(iris_model.parameters(),lr=.01)


In [ ]:
criterion.to(device)
iris_model.to(device)

In [ ]:
optimizer.zero_grad()
loss_=[]
epochs=25
for epoch in range(epochs):
    for x,y in iris_loader:
        print("hi")

        y_pred=iris_model(x.to(device))
        y_pred.shape
        loss=criterion(y_pred.to(device),y.to(device))

        loss_.append(loss)

        with torch.no_grad():
            t,i=torch.max(nn.functional.log_softmax(y_pred.cpu(),dim = 1),1)
            acc=f1_score(y,i,average='weighted')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch}, loss {loss.item()} , accuracy:{acc}")

classification_rep(X_test,y_test)

## real models

In [ ]:
SEED = 1234
torch.backends.cudnn.deterministic = True
torch.manual_seed(SEED)

df = pd.read_excel("train.xlsx")
df = df[['Description', 'label']]
df["desc"] = df["Description"]
df.drop(["Description"], axis=1, inplace=True)
df.columns = ["label", "text"]
df.to_csv('thedata.csv', index=False)


data=pd.read_excel("train.xlsx")
np.random.seed(10)
remove_n=600
drop_indices = np.random.choice(data[data["label"]==2].index, remove_n, replace=False)
data = data.drop(drop_indices)


TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_first=True, tokenize=tokenize_digits)
LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)

dataset = data.TabularDataset(path='thedata.csv', format='csv', skip_header=True,
                              fields=[('label', LABEL), ('text', TEXT)]
                              )

train_dataset, test_dataset = dataset.split(split_ratio=.75)  # default is 0.7
train_dataset, valid_dataset = train_dataset.split(random_state=random.seed(SEED), split_ratio=.90)
len(train_dataset.examples)
len(valid_dataset.examples)
len(test_dataset.examples)

ex = dataset.examples[3]
ex.text
ex.label

train_dataset.examples[0].text
vars(train_dataset[0])

MAX_VOCAB_SIZE = 50000

TEXT.build_vocab(train_dataset,
                 max_size=MAX_VOCAB_SIZE,
                 # vectors="glove.6B.300d",
                 # unk_init=torch.Tensor.normal_,
                 min_freq=10,
                 )

LABEL.build_vocab(train_dataset)

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=False,
    device=device)

for inputs, targets in train_iterator:
    print("imput:", inputs.shape, "shape:", targets.shape)
    break

for batch in valid_iterator:
    print(batch.text)


In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(2, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]

        # text = text.permute(1, 0)

        # text = [batch size, sent len]

        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        # embedded = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim=1))

        # cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)


class RNN(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_hidden, n_rnnlayers, n_outputs):
        super(RNN, self).__init__()
        self.V = n_vocab
        self.D = embed_dim
        self.M = n_hidden
        self.K = n_outputs
        self.L = n_rnnlayers

        self.embed = nn.Embedding(self.V, self.D)
        self.rnn = nn.LSTM(
            input_size=self.D,
            hidden_size=self.M,
            num_layers=self.L,
            batch_first=True)
        self.fc = nn.Linear(self.M, self.K)

    def forward(self, X):
        # initial hidden states
        h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
        c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

        # embedding layer
        # turns word indexes into word vectors
        out = self.embed(X)

        # get RNN unit output
        out, _ = self.rnn(out, (h0, c0))

        # max pool
        out, _ = torch.max(out, 1)

        # we only want h(T) at the final time step
        out = self.fc(out)
        return out


class CNN2(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_outputs):
        super(CNN2, self).__init__()
        self.V = n_vocab
        self.D = embed_dim
        self.K = n_outputs

        # if input is T words
        # then output is (T, D) matrix
        self.embed = nn.Embedding(self.V, self.D)

        # conv layers
        self.conv1 = nn.Conv1d(self.D, 32, 3, padding=1)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(64, 128, 3, padding=1)

        self.fc = nn.Linear(128, self.K)

    def forward(self, X):
        # embedding layer
        # turns word indexes into word vectors
        out = self.embed(X)

        # note: output of embedding is always
        # (N, T, D)
        # conv1d expects
        # (N, D, T)

        # conv layers
        out = out.permute(0, 2, 1)
        out = self.conv1(out)
        out = F.relu(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = F.relu(out)

        # change it back
        out = out.permute(0, 2, 1)

        # max pool
        out, _ = torch.max(out, 1)

        # final dense layer
        out = self.fc(out)
        return out


In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 50
FILTER_SIZES = [2, 3, 4, 5, 6, 7]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
# model = RNN(len(TEXT.vocab), EMBEDDING_DIM, 15, 1, OUTPUT_DIM)
# model = CNN2(len(TEXT.vocab), EMBEDDING_DIM, OUTPUT_DIM)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
optimizer = optim.Adam(model.parameters(), lr=.001)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

N_EPOCHS = 50
best_valid_loss = float('inf')

train_losses = np.zeros(N_EPOCHS)
test_losses = np.zeros(N_EPOCHS)
optimizer.zero_grad()

In [ ]:
for epoch in range(N_EPOCHS):
    optimizer.zero_grad()
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')

    # Save losses
    train_losses[epoch] = train_loss
    test_losses[epoch] = valid_loss

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()

## Another simmilar example with schedueler

In [ ]:
MAX_VOCAB_SIZE = 5000
BATCH_SIZE = 512
MIN_FREQ=5
splt=0.8
tkz=tokenize_plus2
# <editor-fold desc="boiler-plate">
SEED = 1234
torch.backends.cudnn.deterministic = True
torch.manual_seed(SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


df=pd.read_excel("train.xlsx")
np.random.seed(10)
remove_n=600
drop_indices = np.random.choice(df[df["label"]==2].index, remove_n, replace=False)
df = df.drop(drop_indices)
df = df[['Description', 'label']]
df["desc"] = df["Description"]
df.drop(["Description"], axis=1, inplace=True)
df.columns = ["label", "text"]
df.to_csv('thedata-notrip.csv', index=False)

tokenizer = get_tokenizer('basic_english')
TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_first=True, tokenize=tkz)
LABEL = data.Field(sequential=False, use_vocab=False, is_target=True)
dataset = data.TabularDataset(path='thedata-notrip.csv', format='csv', skip_header=True,
                              fields=[('label', LABEL), ('text', TEXT)]
                              )
train_dataset, test_dataset = dataset.split(split_ratio=splt)  # default is 0.7
train_dataset, valid_dataset = train_dataset.split(random_state=random.seed(SEED), split_ratio=splt)
len(dataset)


TEXT.build_vocab(train_dataset,
                 max_size=MAX_VOCAB_SIZE,
                 # vectors="glove.6B.300d",
                 # unk_init=torch.Tensor.normal_,
                 min_freq=MIN_FREQ
                 )

LABEL.build_vocab(train_dataset)



train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    # sort_key=lambda x: len(x.text),
    sort_within_batch=False,
    sort=False,
    device=device)


e=nn.Embedding(5, 4)
t=np.random.randint(1,5,[3,2])
tt=torch.LongTensor(t)
e(tt)#3x2 ---> 3x2x4
# </editor-fold>
DROPOUT = 0.5




NameError: name 'tokenize_plus2' is not defined

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=n_filters,
                      kernel_size=(2, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]

        # text = text.permute(1, 0)

        # text = [batch size, sent len]

        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        # embedded = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim=1))

        # cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)

class RNN(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_hidden, n_rnnlayers, n_outputs,p=.5):
        super(RNN, self).__init__()
        self.V = n_vocab
        self.D = embed_dim
        self.M = n_hidden
        self.K = n_outputs
        self.L = n_rnnlayers

        self.dropout=nn.Dropout(p=p)

        self.embed = nn.Embedding(self.V, self.D)
        self.rnn = nn.LSTM(
            input_size=self.D,
            hidden_size=self.M,
            num_layers=self.L,
            batch_first=True)
        self.fc = nn.Linear(self.M, self.K)

    def forward(self, X):
        # initial hidden states
        h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
        c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

        # embedding layer
        # turns word indexes into word vectors
        out = self.embed(X)

        # get RNN unit output
        out, _ = self.rnn(out, (h0, c0))

        out=self.dropout(out)

        # max pool
        out, _ = torch.max(out, 1)

        # we only want h(T) at the final time step
        out = self.fc(out)
        return out

class CNN2(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_outputs):
        super(CNN2, self).__init__()
        self.V = n_vocab
        self.D = embed_dim
        self.K = n_outputs

        # if input is T words
        # then output is (T, D) matrix
        self.embed = nn.Embedding(self.V, self.D)

        # conv layers
        self.conv1 = nn.Conv1d(self.D, 32, 3, padding=1)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(64, 128, 3, padding=1)

        self.fc = nn.Linear(128, self.K)

    def forward(self, X):
        # embedding layer
        # turns word indexes into word vectors
        out = self.embed(X)

        # note: output of embedding is always
        # (N, T, D)
        # conv1d expects
        # (N, D, T)

        # conv layers
        out = out.permute(0, 2, 1)
        out = self.conv1(out)
        out = F.relu(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = F.relu(out)

        # change it back
        out = out.permute(0, 2, 1)

        # max pool
        out, _ = torch.max(out, 1)

        # final dense layer
        out = self.fc(out)
        return out

class CNN3(nn.Module):
    def __init__(self, n_vocab, embed_dim, n_outputs,p=.1):
        super(CNN3, self).__init__()
        self.V = n_vocab
        self.D = embed_dim
        self.K = n_outputs

        # if input is T words
        # then output is (T, D) matrix
        self.embed = nn.Embedding(self.V, self.D)
        self.dropout=nn.Dropout(p)

        # conv layers
        self.conv1 = nn.Conv1d(self.D, 64, 3, padding=1)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, 3, padding=1)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(128, 64, 3, padding=1)

        self.fc = nn.Linear(64, self.K)

    def forward(self, X):
        # embedding layer
        # turns word indexes into word vectors
        out = self.embed(X)

        # note: output of embedding is always
        # (N, T, D)
        # conv1d expects
        # (N, D, T)

        # conv layers
        out = out.permute(0, 2, 1)
        out = self.conv1(out)
        out = self.dropout(F.relu(out))
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.dropout(F.relu(out))
        out = self.pool2(out)
        out = self.conv3(out)
        out = F.relu(out)

        # change it back
        out = out.permute(0, 2, 1)

        # max pool
        out, _ = torch.max(out, 1)

        # final dense layer
        out = self.fc(out)
        return out

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 50
FILTER_SIZES = [2, 3, 4, 5, 6, 7]
OUTPUT_DIM = len(LABEL.vocab)

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
LR=.0005

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
# model = CNN2(len(TEXT.vocab), EMBEDDING_DIM, OUTPUT_DIM)
# model = RNN(len(TEXT.vocab), EMBEDDING_DIM, 15, 1, OUTPUT_DIM,p=DROPOUT)
# model = CNN3(len(TEXT.vocab), EMBEDDING_DIM, OUTPUT_DIM,p=DROPOUT)

model.to(device)

print(f'The model has {count_parameters(model):,} trainable parameters')
optimizer = optim.Adam(model.parameters(), lr=LR,weight_decay=1e-4)
scheduler=ReduceLROnPlateau(optimizer,"min")
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)
N_EPOCHS = 100
best_valid_loss = float('inf')

train_losses = np.zeros(N_EPOCHS)
test_losses = np.zeros(N_EPOCHS)
optimizer.zero_grad()

In [ ]:
for epoch in range(N_EPOCHS):
    optimizer.zero_grad()
    start_time = time.time()

    train_loss, train_acc = train_(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    scheduler.step(valid_loss)


    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')

    # Save losses
    train_losses[epoch] = train_loss
    test_losses[epoch] = valid_loss

    print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

test_loss, test_acc = evaluate(model, test_iterator, criterion)


predictions=classification_rep(X_test,y_test)
plot_confusion_matrix_(data,predictions)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()